In [1]:
%run common.ipynb

In [2]:
from pyspark.sql import SparkSession
import random

# Initialize Spark Session with MySQL JDBC Driver
spark = SparkSession.builder.appName("RetailCustomer360") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c7a12a1c-1f6f-4615-9922-7efb03b85b72;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
:: resolution report :: resolve 322ms :: artifacts dl 8ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------------------------------------

In [3]:
# Read MySQL table into PySpark DataFrame
customer_df = spark.read.jdbc(url=USER_MYSQL_URL, table="Customers", properties=MYSQL_PROPERTIES)
order_df = spark.read.jdbc(url=ORDER_MYSQL_URL, table="Orders", properties=MYSQL_PROPERTIES)
products_df = spark.read.jdbc(url=PRODUCT_MYSQL_URL, table="Products", properties=MYSQL_PROPERTIES)
# Register as Temp Tables
customer_df.createOrReplaceTempView("customers")
order_df.createOrReplaceTempView("orders")


In [4]:
customer_df.show()

+-----------+----------+---------+--------------------+--------------------+--------------------+-------------------+--------------+--------+--------------------+-------------------+
|customer_id|first_name|last_name|               email|               phone|             address|               city|         state|zip_code|             country|         created_at|
+-----------+----------+---------+--------------------+--------------------+--------------------+-------------------+--------------+--------+--------------------+-------------------+
|          1|   Charles|    Jones|rodneysullivan@ex...| (337)512-3234x99488|365 Ricky Spur\nK...|         Jacobmouth|        Kansas|   72281|United States of ...|2025-03-17 03:54:54|
|          2|   Brandon|  Mcgrath|crystalnavarro@ex...|  953.750.7854x37299|6373 Susan Inlet ...|       Chambersview|      Oklahoma|   20216|          Luxembourg|2025-03-17 03:54:51|
|          3|     Misty|   Warren| fpineda@example.com|   315.763.1867x5725|67991 Mic

In [5]:
spark.sql("""
        SELECT *   FROM orders o  where o.customer_id=553 """).show(100)

+--------+-----------+-------------------+---------+------------+
|order_id|customer_id|         order_date|   status|total_amount|
+--------+-----------+-------------------+---------+------------+
|    2256|        553|2025-01-05 00:00:00|Cancelled|      392.96|
|   12065|        553|2020-06-01 00:00:00|  Pending|      275.97|
|   12586|        553|2023-08-31 00:00:00|  Shipped|       87.41|
|   19939|        553|2022-07-25 00:00:00|Cancelled|       28.39|
|   20799|        553|2022-07-18 00:00:00|  Pending|      388.93|
|   24883|        553|2020-05-02 00:00:00|Delivered|      289.27|
|   29740|        553|2023-09-15 00:00:00|  Pending|      104.74|
|   31874|        553|2023-02-09 00:00:00|  Pending|      229.42|
|   41874|        553|2020-04-12 00:00:00|  Pending|      161.43|
|   56057|        553|2023-09-11 00:00:00|  Shipped|      296.85|
|   57301|        553|2021-06-27 00:00:00|Delivered|       80.25|
|   59242|        553|2024-11-16 00:00:00|Delivered|      441.85|
|   61722|

In [6]:
spark.sql("""
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent
        FROM orders o GROUP BY o.customer_id  order by total_spent desc""").show()

+-----------+-----------+
|customer_id|total_spent|
+-----------+-----------+
|       2333|    7776.53|
|       9408|    6713.81|
|        949|    6480.47|
|       2171|    6462.36|
|       6018|    6254.54|
|       6864|    6232.70|
|       5758|    6159.81|
|       7130|    6135.97|
|       1643|    6131.51|
|       6056|    6064.61|
|       1439|    6013.41|
|       4027|    5953.78|
|       3294|    5952.49|
|       5210|    5848.03|
|       1494|    5830.34|
|       2350|    5820.18|
|        402|    5814.20|
|       3960|    5780.35|
|       2601|    5751.75|
|        545|    5742.82|
+-----------+-----------+
only showing top 20 rows



In [7]:
top_customers = spark.sql("""
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent,
            COUNT(o.order_id) AS total_orders,
            MAX(o.order_date) AS last_purchase_date
        FROM orders o 
        WHERE o.order_date >= date_add(current_date(), -365)  -- Last 1 year
        GROUP BY o.customer_id order by total_spent desc  """).show(10)

+-----------+-----------+------------+-------------------+
|customer_id|total_spent|total_orders| last_purchase_date|
+-----------+-----------+------------+-------------------+
|       1531|    2969.88|           8|2025-02-14 00:00:00|
|       5386|    2794.61|           8|2025-03-13 00:00:00|
|         47|    2554.18|           7|2024-11-09 00:00:00|
|       6957|    2497.63|           7|2025-03-03 00:00:00|
|       4144|    2429.91|           6|2025-01-19 00:00:00|
|       2601|    2339.36|           8|2025-03-09 00:00:00|
|       9258|    2293.19|           7|2025-03-01 00:00:00|
|       9405|    2277.66|           7|2024-12-24 00:00:00|
|       1456|    2267.00|           7|2025-01-13 00:00:00|
|       7315|    2263.81|           6|2025-01-27 00:00:00|
+-----------+-----------+------------+-------------------+
only showing top 10 rows



In [8]:
 spark.sql("""
    WITH customer_spending AS (
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent,
            COUNT(o.order_id) AS total_orders,
            MAX(o.order_date) AS last_purchase_date
        FROM orders o
        WHERE o.order_date >= date_add(current_date(), -365)   -- Last 1 year
        GROUP BY o.customer_id order by total_spent desc 
    )
        SELECT
            c.country,
            c.customer_id,
            c.first_name,
            c.email,
            cs.total_spent,
            cs.total_orders,
            cs.last_purchase_date,
            RANK() OVER ( PARTITION BY c.country ORDER BY cs.total_spent DESC) AS spending_rank
        FROM customer_spending cs
        JOIN customers c ON cs.customer_id = c.customer_id where c.country='United Kingdom'  GROUP BY c.country,c.customer_id,c.first_name,c.email,cs.total_spent,cs.last_purchase_date,cs.total_orders order by total_spent desc
""").show(100)


+--------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|       country|customer_id|first_name|               email|total_spent|total_orders| last_purchase_date|spending_rank|
+--------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|United Kingdom|       7895|   Kenneth|  pestes@example.org|    1693.16|           5|2024-11-24 00:00:00|            1|
|United Kingdom|        545|  Lawrence|chopkins@example.org|    1490.65|           5|2025-03-09 00:00:00|            2|
|United Kingdom|       3265|   Shannon|kathrynmccarthy@e...|    1404.09|           6|2025-02-24 00:00:00|            3|
|United Kingdom|       5438|    Edward|dorothywalker@exa...|    1368.42|           6|2025-01-17 00:00:00|            4|
|United Kingdom|       9793|   Anthony|justin90@example.com|    1283.33|           4|2025-02-22 00:00:00|            5|
|United Kingdom|        591|    Nicole|s

In [9]:

# Run Spark SQL Query
top_customers = spark.sql("""
    WITH customer_spending AS (
        SELECT
            o.customer_id,
            SUM(o.total_amount) AS total_spent,
            COUNT(o.order_id) AS total_orders,
            MAX(o.order_date) AS last_purchase_date
        FROM orders o
        WHERE o.order_date >= date_add(current_date(), -365)  -- Last 1 year
        GROUP BY o.customer_id
    ),
    customer_ranking AS (
        SELECT
            c.country,
            c.customer_id,
            c.first_name,
            c.email,
            cs.total_spent,
            cs.total_orders,
            cs.last_purchase_date,
            RANK() OVER (PARTITION BY c.country ORDER BY cs.total_spent DESC) AS spending_rank
        FROM customer_spending cs
        JOIN customers c ON cs.customer_id = c.customer_id
    )
    SELECT * FROM customer_ranking WHERE spending_rank <= 10 and country like 'United %';
""")

# Show results
top_customers.show(100)


+--------------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|             country|customer_id|first_name|               email|total_spent|total_orders| last_purchase_date|spending_rank|
+--------------------+-----------+----------+--------------------+-----------+------------+-------------------+-------------+
|United Arab Emirates|       3222|    George|jacobpeterson@exa...|    1487.63|           4|2024-10-06 00:00:00|            1|
|United Arab Emirates|       9505|     Tonya|medinalisa@exampl...|    1149.79|           4|2025-02-03 00:00:00|            2|
|United Arab Emirates|        752|   William|josebradford@exam...|    1111.42|           3|2024-08-24 00:00:00|            3|
|United Arab Emirates|       3814|   Krystal|jbradley@example.net|    1080.19|           3|2025-02-24 00:00:00|            4|
|United Arab Emirates|       6528|   Tabitha|  rlewis@example.net|     974.51|           4|2025-03-06 00:00:00|       

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


# Filter last 1 year of data
one_year_ago = F.date_add(F.current_date(), -365)
filtered_orders = order_df.filter(F.col("order_date") >= one_year_ago)

# Aggregate customer spending
customer_spending = (
    filtered_orders.groupBy("customer_id")
    .agg(
        F.sum("total_amount").alias("total_spent"),
        F.count("order_id").alias("total_orders"),
        F.max("order_date").alias("last_purchase_date")
    )
)

# Join with customers table
customer_data = customer_spending.join(customer_df, "customer_id")

# Define window specification for ranking
window_spec = Window.partitionBy("country").orderBy(F.desc("total_spent"))

# Add ranking column
customer_ranking = customer_data.withColumn("spending_rank", F.rank().over(window_spec))

# Filter top 100 customers
top_customers = customer_ranking.filter( (F.col("spending_rank") <= 10) &  (F.col("country").like("United %")))

# Show results
top_customers.select("country","customer_id","first_name","email","total_spent","total_orders","spending_rank").show(20)

+--------------------+-----------+----------+--------------------+-----------+------------+-------------+
|             country|customer_id|first_name|               email|total_spent|total_orders|spending_rank|
+--------------------+-----------+----------+--------------------+-----------+------------+-------------+
|United Arab Emirates|       3222|    George|jacobpeterson@exa...|    1487.63|           4|            1|
|United Arab Emirates|       9505|     Tonya|medinalisa@exampl...|    1149.79|           4|            2|
|United Arab Emirates|        752|   William|josebradford@exam...|    1111.42|           3|            3|
|United Arab Emirates|       3814|   Krystal|jbradley@example.net|    1080.19|           3|            4|
|United Arab Emirates|       6528|   Tabitha|  rlewis@example.net|     974.51|           4|            5|
|United Arab Emirates|       7612|     Roger| lnelson@example.net|     954.26|           4|            6|
|United Arab Emirates|       9402|    Lauren| 

In [11]:
# Define AWS Glue database and table names
glue_database = "customer_analytics"
glue_table = "top_100_customers"

# Define S3 output path
s3_output_path = "s3://feb2025-training-bucket/analytics/top_100_customers/"

# Create the AWS Glue Catalog table using the DataFrame
top_customers.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", s3_output_path) \
    .saveAsTable(f"{glue_database}.{glue_table}")

print(f"Aggregated sales data written to S3: {s3_output_path}")
print(f"Glue table '{glue_database}.{glue_table}' created successfully.")


25/03/17 04:01:47 INFO HiveConf: Found configuration file file:/home/glue_user/spark/conf/hive-site.xml
25/03/17 04:01:49 WARN InstanceMetadataServiceResourceFetcher: Fail to retrieve token 
com.amazonaws.SdkClientException: Failed to connect to service endpoint: 
	at com.amazonaws.internal.EC2ResourceFetcher.doReadResource(EC2ResourceFetcher.java:100)
	at com.amazonaws.internal.InstanceMetadataServiceResourceFetcher.getToken(InstanceMetadataServiceResourceFetcher.java:91)
	at com.amazonaws.internal.InstanceMetadataServiceResourceFetcher.readResource(InstanceMetadataServiceResourceFetcher.java:69)
	at com.amazonaws.internal.EC2ResourceFetcher.readResource(EC2ResourceFetcher.java:66)
	at com.amazonaws.util.EC2MetadataUtils.getItems(EC2MetadataUtils.java:407)
	at com.amazonaws.util.EC2MetadataUtils.getData(EC2MetadataUtils.java:376)
	at com.amazonaws.util.EC2MetadataUtils.getData(EC2MetadataUtils.java:372)
	at com.amazonaws.util.EC2MetadataUtils.getEC2InstanceRegion(EC2MetadataUtils.java

Aggregated sales data written to S3: s3://feb2025-training-bucket/analytics/top_100_customers/
Glue table 'customer_analytics.top_100_customers' created successfully.
